In [26]:
import descarteslabs as dl
import numpy as np

For context, one of our platform users, Hugo, is seeing all-zero arrays come back where they are not expected. I have recreated the issue here, with a few alternative queries to demonstrate the expected behavior, vs the output of his queries. His parecel of land, defined as "aoi" below is small, around 70m x 100m - but the resolution of the data is 10m. 

In [1]:
aoi = {"coordinates": [[[-9.0915298, 39.3650692], 
                        [-9.0915513, 39.3645093], 
                        [-9.090988, 39.3645342], 
                        [-9.0909773, 39.3649157], 
                        [-9.0909076, 39.3649489], 
                        [-9.0909022, 39.3651148], 
                        [-9.0915298, 39.3650692]]], 
       "type": "Polygon"}

In [28]:
taos = {'coordinates':[[[-105.8898318, 38.5103536],
                        [-103.8445201, 38.1129959],
                        [-104.3838427, 36.4246539],
                        [-106.377808, 36.8201224],
                        [-105.8898318, 38.5103536]]],
       'type': 'Polygon'}

In [23]:
features = dl.metadata.search(
    products='sentinel-3:OLCI_RGB',
    start_time='2017-01-01',
    end_time='2017-03-31',
    limit=80,
    geom=aoi)

print(len(features['features']))


1


This is the output for data for the entire first scene covering Hugo's parcel - there is valid data there.  

In [25]:
for i in features['features']:
    arr, meta = dl.raster.ndarray(
        i['id'],
        bands=['blue'],
        scales=[[0, 4000]],
        data_type='Byte',
        resolution=10,)
#         cutline=aoi)
   
    print(i['id'])
    print(np.unique(arr, return_counts=True))
    print(arr.shape)
    print("_______________")
    


sentinel-3:OLCI_RGB:meta_2017-01-02-1115_29N_07_S3A_v0
(array([  0,   1,   2,   3,  48,  49,  50,  51,  52,  53,  54,  55,  56,
        57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
        70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,
        83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,
        96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108,
       109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121,
       122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134,
       135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147,
       148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160,
       161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173,
       174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186,
       187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199,
       200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212,
       2

The baffling query, same as above, but greater time range and rasterized with a cutline of the origin search geometry. Why are all returned arrays zero???

In [31]:
features = dl.metadata.search(
    products='sentinel-3:OLCI_RGB',
    start_time='2017-01-01',
    end_time='2017-03-31',
    limit=80,
    geom=aoi)

print(len(features['features']))

for i in features['features']:
    arr, meta = dl.raster.ndarray(
        i['id'],
        bands=['blue'],
        scales=[[0, 4000]],
        data_type='Byte',
        resolution=10,
        cutline=aoi)
   
    print(i['id'])
    print(np.unique(arr, return_counts=True))
    print(arr.shape)
    print("_______________")
    

60
sentinel-3:OLCI_RGB:meta_2017-01-02-1115_29N_07_S3A_v0
(array([0], dtype=uint8), array([42]))
(7, 6)
_______________
sentinel-3:OLCI_RGB:meta_2017-01-03-1049_29N_07_S3A_v0
(array([0], dtype=uint8), array([42]))
(7, 6)
_______________
sentinel-3:OLCI_RGB:meta_2017-01-04-1023_29N_07_S3A_v0
(array([0], dtype=uint8), array([42]))
(7, 6)
_______________
sentinel-3:OLCI_RGB:meta_2017-01-06-1111_29N_07_S3A_v0
(array([0], dtype=uint8), array([42]))
(7, 6)
_______________
sentinel-3:OLCI_RGB:meta_2017-01-07-1045_29N_07_S3A_v0
(array([0], dtype=uint8), array([42]))
(7, 6)
_______________
sentinel-3:OLCI_RGB:meta_2017-01-08-1019_29N_07_S3A_v0
(array([0], dtype=uint8), array([42]))
(7, 6)
_______________
sentinel-3:OLCI_RGB:meta_2017-01-10-1108_29N_07_S3A_v0
(array([0], dtype=uint8), array([42]))
(7, 6)
_______________
sentinel-3:OLCI_RGB:meta_2017-01-11-1042_29N_07_S3A_v0
(array([0], dtype=uint8), array([42]))
(7, 6)
_______________
sentinel-3:OLCI_RGB:meta_2017-01-12-1015_29N_07_S3A_v0
(array

Print values for Taos geometry using similar query as sanity check. Note that there are two images returned with all-zero arrays. 

In [30]:
features = dl.metadata.search(
    products='sentinel-3:OLCI_RGB',
    start_time='2017-01-01',
    end_time='2017-01-31',
    limit=80,
    geom=taos)

print(len(features['features']))

for i in features['features']:
    arr, meta = dl.raster.ndarray(
        i['id'],
        bands=['blue'],
        scales=[[0, 4000]],
        data_type='Byte',
        resolution=10,
        cutline=taos)
   
    print(i['id'])
    print(np.unique(arr, return_counts=True))
    print(arr.shape)
    print("_______________")
    

22
sentinel-3:OLCI_RGB:meta_2017-01-01-1644_13N_07_S3A_v0
(array([  0,   1,   2,   3,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181,
       182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194,
       195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207,
       208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220,
       221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233,
       234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246,
     

sentinel-3:OLCI_RGB:meta_2017-01-05-1641_13N_07_S3A_v0
(array([  0,   1,   2,   3, 255], dtype=uint8), array([431362822,    270864,      8352,       144,  87536280]))
(23159, 22418)
_______________
sentinel-3:OLCI_RGB:meta_2017-01-07-1729_13N_07_S3A_v0
(array([  0,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,
        80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
        93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105,
       106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
       119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131,
       132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144,
       145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157,
       158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170,
       171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183,
       184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196,
       197, 198, 199, 200,

sentinel-3:OLCI_RGB:meta_2017-01-12-1659_13N_07_S3A_v0
(array([  0,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100,
       101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113,
       114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126,
       127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
       140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152,
       153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165,
       166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178,
       179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191,
       192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204,
       205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217,
       218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230,
       231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243,
       244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255],
      dtype=

sentinel-3:OLCI_RGB:meta_2017-01-19-1718_13N_07_S3A_v0
(array([  0,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182,
       183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195,
       196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208,
       209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,
       2

sentinel-3:OLCI_RGB:meta_2017-01-23-1714_13N_07_S3A_v0
(array([  0,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,
        69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,
        82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,
        95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107,
       108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120,
       121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133,
       134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146,
       147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159,
       160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172,
       173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185,
       186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198,
       199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211,
       212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224,
       2

sentinel-3:OLCI_RGB:meta_2017-01-27-1710_13N_07_S3A_v0
(array([  0,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,
        71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
        84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,
        97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109,
       110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122,
       123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135,
       136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148,
       149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161,
       162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174,
       175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187,
       188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200,
       201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213,
       214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226,
       2